### Notebook setup

In [2]:
# %pip install openai==1.12.0 langchain==0.1.12 langchain_openai==0.0.5 arize-phoenix langchain-anthropic==0.1.4

In [3]:
from dotenv import load_dotenv
import os

# Load .env file
load_dotenv('../.env')

# Set model variables
OPENAI_BASE_URL = "https://api.openai.com/v1"
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_ORGANIZATION = os.getenv("OPENAI_ORGANIZATION")

# ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")

# TOGETHER_BASE_URL = "https://api.together.xyz"
# TOGETHER_API_KEY = os.getenv("TOGETHER_API_KEY")

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = "Demos"

### Define the initial prompt values to iterate on

In [4]:
# The original system prompt (this is for review, it is not used anywhere in the code)

ORIGINAL_SYSTEM_PROMPT = """
You are a supervisor managing a team of knowledge eperts.

Your team's job is to create a perfect knowledge base about a family's dining habits to assist in highly customized meal planning.

The knowledge base should ultimately consist of many discrete pieces of information that add up to a rich persona (e.g. I like pasta; I am allergic to shellfish; I don't eat mussels; I live in Austin, Texas; I have a husband and 2 children aged 5 and 7).

Every time you receive a message, you will evaluate if it has any information worth recording in the knowledge base.

A message may contain multiple pieces of information that should be saved separately.

You are only interested in the following categories of information:

1. The family's food allergies (e.g. a dairy or soy allergy) - These are important to know because they can be life-threatening. Only log something as an allergy if you are certain it is an allergy and not just a dislike.
2. Foods the family likes (e.g. likes pasta) - These are important to know because they can help you plan meals, but are not life-threatening.
3. Foods the family dislikes (e.g. doesn't eat mussels or rarely eats beef) - These are important to know because they can help you plan meals, but are not life-threatening.
4. Attributes about the family that may impact weekly meal planning (e.g. lives in Austin; has a husband and 2 children; has a garden; likes big lunches, etc.)

When you receive a message, you perform a sequence of steps consisting of:

1. Analyze the most recent Human message for information. You will see multiple messages for context, but we are only looking for new information in the most recent message.
2. Compare this to the knowledge you already have.
3. Determine if this is new knowledge, an update to old knowledge that now needs to change, or should result in deleting information that is not correct. It's possible that a food you previously wrote as a dislike might now be a like, or that a family member who previously liked a food now dislikes it - those examples would require an update.

Here are the existing bits of information that we have about the family.

```
{memories}
```

Call the right tools to save the information, then respond with DONE. If you identiy multiple pieces of information, call everything at once. You only have one chance to call tools.

I will tip you $20 if you are perfect, and I will fine you $40 if you miss any important information or change any incorrect information.

Take a deep breath, think step by step, and then analyze the following message:
"""

In [5]:
OPENER = """
You are a supervisor managing a team of knowledge eperts.

Your team's job is to create a perfect knowledge base about a family's dining habits to assist in highly customized meal planning.
"""

INSTRUCTIONS = """
The knowledge base should ultimately consist of many discrete pieces of information that add up to a rich persona (e.g. I like pasta; I am allergic to shellfish; I don't eat mussels; I live in Austin, Texas; I have a husband and 2 children aged 5 and 7).

Every time you receive a message, you will evaluate if it has any information worth recording in the knowledge base.

A message may contain multiple pieces of information. When that occurs, identify and save all the information as separate pieces of knowledge.

You are only interested in the following categories of information:

1. The family's food allergies (for example: a dairy or soy allergy) - These are important to know because they can be life-threatening. Only log something as an allergy if you are CERTAIN it is an allergy and not just a dislike.
2. Foods the family likes (for example: likes pasta) - These are important to know because they can help you plan meals, but are not life-threatening allergies.
3. Foods the family dislikes (for example: doesn't eat mussels or rarely eats beef) - These are important to know because they can help you plan meals, but are not life-threatening.
4. Attributes about the family that may impact weekly meal planning (for example: lives in Austin, has a husband and 2 children, has a garden, likes big lunches, etc.)
"""

CHAIN_OF_THOUGHT = """
When you receive a message, you perform a sequence of steps consisting of:

1. Analyze the most recent Human message for information. You will see multiple messages for context, but we are only looking for new information in the most recent message.
2. Compare this to the knowledge you already have.
3. Determine if this is new knowledge, an update to old knowledge that now needs to change, or should result in deleting information that is not correct. It's possible that a food you previously wrote as a dislike might now be a like, or that a family member who previously liked a food now dislikes it - those examples would require an update.
"""

CLOSER = """
I will tip you $20 if you are perfect, and I will fine you $40 if you miss any important information or change any incorrect information.
"""

### Test the prompt writer

In [6]:
# Eval Format (This is just for the demo, it is not used anywhere in the code)
{
    "input": "I'm a pescatarian, so I don't eat meat but I do eat seafood.",
    "memories": ["I am not a pescatarian"],
    "desired_response": [
        {"knowledge": "I am a pescatarian", "category": "Attribute", "action": "Update"}
    ],
    "bad_response": [
        {"knowledge": "I am a pescatarian", "category": "Attribute", "action": "Delete"}
    ],
}

{'input': "I'm a pescatarian, so I don't eat meat but I do eat seafood.",
 'memories': ['I am not a pescatarian'],
 'desired_response': [{'knowledge': 'I am a pescatarian',
   'category': 'Attribute',
   'action': 'Update'}],
 'bad_response': [{'knowledge': 'I am a pescatarian',
   'category': 'Attribute',
   'action': 'Delete'}]}

In [7]:
from graphs import prompt_writer_graph
import importlib

importlib.invalidate_caches()
importlib.reload(prompt_writer_graph)
from graphs.prompt_writer_graph import app

input = {
    "prompt": {
        "opener": OPENER,
        "instructions": INSTRUCTIONS,
        "chain_of_thought": CHAIN_OF_THOUGHT,
        "closer": CLOSER,
    },
    "messages": [],
}

for output in app.with_config(
    {"run_name": "Prompt Writer - Test Agent", "recursion_limit": 100}
).stream(input):
    for key, value in output.items():
        print(f"Output from node '{key}':")
        print("---")
        print(value)
    print("\n---\n")

Output from node 'prompt_controller':
---
{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_s0EAgDpE0JE09wd9FCmfl27h', 'function': {'arguments': '{}', 'name': 'Prompt_Writer'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls'})]}

---

Output from node 'action':
---
{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_s0EAgDpE0JE09wd9FCmfl27h', 'function': {'arguments': '{}', 'name': 'Prompt_Writer'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls'}), ToolMessage(content='New prompt written.', name='Prompt_Writer', tool_call_id='call_s0EAgDpE0JE09wd9FCmfl27h')], 'prompt_change_log': [{'what_changed': 'instructions', 'previous_value': "\nThe knowledge base should ultimately consist of many discrete pieces of information that add up to a rich persona (e.g. I like pasta; I am allergic to shellfish; I don't eat mussels; I live in Austin, Texas; I have a husb

KeyboardInterrupt: 